## This file get info already stored in chroma database (the best version)

### Import the libraries to get the env variables

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_VERSION = "2023-03-15-preview"

In [3]:
OPENAI_API_KEY 

'9ac347d13e834f288a2076ff9c7b418a'

In [18]:
# database = "./conventia_viena-database"
# database = "C:/Users/Alex/OneDrive/Documente/PersonalRepos/AI/LangChain/AzureCognitiveSearch/barack_obama-database"
# database = "C:/Users/uif94707/Documents/myProjects/School-App/School-App/server/electrotehnica-database"
database = "./electrotehnica-database"
query = "care este defazajul dintre tensiune şi curent?"

### Import the libraries to connect to chroma database

In [19]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from chromadb.utils import embedding_functions


In [20]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


##### Connect to chroma database

In [21]:
db3 = Chroma(persist_directory=database, embedding_function=embedding_function)
db3

##### Create the retriever and get relevant documents

In [22]:
retriever = db3.as_retriever(search_type="similarity", search_kwargs = {"k": 2})
docs = retriever.get_relevant_documents(query)

len(docs)
print(docs)


[Document(page_content='Modulul impedanţei complexe \nL X ZL\uf077==  - se numeşte reactanţa inductivă. Tensiunea la \nbornele bobinei este defazată cu \n2π=\uf06a  în faţa curentului.  \n \n \nCondensatorul ideal   \nCondensatorul ideal este caracterizat de capacitatea electrică C. Ecuaţia de funcţionare \neste: \n\uf0f2\uf0d7\uf0d7= dti(t)C1u(t)\n  \nEcuaţia de funcţionare în complex este:  \nICjω1U \uf0d7\uf0d7=\n  \nImpedanţa complexă:  \nCjω1\n\uf0d7=Z\n \nCω1\n\uf0d7=Z\n - se numeşte reactanţă capacitivă.  \nDefazajul dintre tensiune şi curent este:\n2π−=\uf06a . \nFigura 5.10', metadata={'page': 1, 'source': 'electrotehnica_documents\\curs1.pdf'}), Document(page_content='ELEMENTELE IDEALE DE CIRCUIT ÎN REGIM SINUSOIDAL  \n \n Rezistorul ideal   \nRezistorul ideal prezintă o rezistenţă electrică R şi este caracterizat de ecuaţia:  \n)( u(t) tRi=\n \nEcuaţia de funcţionare în complex devine:  \nIRU=\n \nImpedanţa complexă a rezistorului va fi:  \nIUZ=\n  \nDefazajul dintre tensiun

#### Import the model 3.5 turbo

In [23]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA

llm = AzureChatOpenAI(
    deployment_name = "gpt35-uif54579",
    model_name = "gpt-35-turbo",
    temperature = 0.5
)

In [24]:
llm

AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.5, model_kwargs={}, openai_api_key='9ac347d13e834f288a2076ff9c7b418a', openai_api_base='https://sbzdfopenai.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='gpt35-uif54579', model_version='', openai_api_type='azure', openai_api_version='2023-03-15-preview')

In [25]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever = retriever, return_source_documents = True)


In [47]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print("\n\nSources:")
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [26]:
llm_response = chain(query)
llm_response

{'query': 'care este defazajul dintre tensiune şi curent?',
 'result': 'Pentru bobina ideală, defazajul dintre tensiune şi curent este de 2π=\uf06a în faţa curentului.',
 'source_documents': [Document(page_content='Modulul impedanţei complexe \nL X ZL\uf077==  - se numeşte reactanţa inductivă. Tensiunea la \nbornele bobinei este defazată cu \n2π=\uf06a  în faţa curentului.  \n \n \nCondensatorul ideal   \nCondensatorul ideal este caracterizat de capacitatea electrică C. Ecuaţia de funcţionare \neste: \n\uf0f2\uf0d7\uf0d7= dti(t)C1u(t)\n  \nEcuaţia de funcţionare în complex este:  \nICjω1U \uf0d7\uf0d7=\n  \nImpedanţa complexă:  \nCjω1\n\uf0d7=Z\n \nCω1\n\uf0d7=Z\n - se numeşte reactanţă capacitivă.  \nDefazajul dintre tensiune şi curent este:\n2π−=\uf06a . \nFigura 5.10', metadata={'page': 1, 'source': 'electrotehnica_documents\\curs1.pdf'}),
  Document(page_content='ELEMENTELE IDEALE DE CIRCUIT ÎN REGIM SINUSOIDAL  \n \n Rezistorul ideal   \nRezistorul ideal prezintă o rezistenţă elec

In [27]:
process_llm_response(llm_response)

NameError: name 'process_llm_response' is not defined

In [28]:
llm_response['result']

'Pentru bobina ideală, defazajul dintre tensiune şi curent este de 2π=\uf06a în faţa curentului.'